# Stage 1 Model Selection: Summary of Results

Model selection was conducted using a method of time series cross-validation called rolling forecast origin. To avoid leakage of future observations, the method incrementally moves the forecast origin forward in time and then makes a prediction. For each new fold we implemented a stride of seven days. We performed a two stage model selection procedure. In the first stage, we used an aggregate regional level time series to screen and identify the most promising candidate models for up to a 365 day forecast (27 folds).

This notebook generates the summary result tables and charts for stage 1 of model selection (screening)


In [1]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

In [2]:
cwd = os.getcwd()
cwd

'/home/tom/Documents/code/swast-benchmarking/results'

In [27]:
if cwd[-7:] != "results":
    mypath = './results/model_selection/stage1/'
    TABLE_PATH = './paper/tables/'
    FIGURE_PATH = './paper/figures/'
    APPENDIX_PATH = './paper/appendix/'
else:
    mypath = './model_selection/stage1/'
    TABLE_PATH = '../paper/tables/'
    FIGURE_PATH = '../paper/figures/'
    FIGURE_PATH = '../paper/appendix/'

In [4]:
result_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [5]:
results_mean = pd.DataFrame()
results_med = pd.DataFrame()
results_mean_std = pd.DataFrame()

## Point Estimate Results

In [6]:
error_measures = ['smape', 'rmse', 'mase', 'coverage_80', 'coverage_95']

In [7]:
for metric in error_measures:
    to_read = [filename for filename in result_files if metric in filename]
    model_names = [name[:name.index('_')] for name in to_read]
    
    for filename, model_name in zip(to_read, model_names):
        df = pd.read_csv(mypath + filename, index_col=0)

        prefix = model_name + '_' + metric
        results_mean[prefix + '_mean'] = df.mean()
        results_mean[prefix  + '_std'] = df.std()
        results_med[prefix + '_med'] = df.median()
        results_med[prefix + '_iqr'] = df.quantile(0.75) - df.quantile(0.25)
        
        results_mean_std[prefix] = results_mean[prefix + '_mean'].map('{:,.2f}'.format) \
            + ' (' + results_mean[prefix  + '_std'].map('{:,.2f}'.format) + ')'

In [8]:
results_mean.filter(like="fbp")

,fbp-hw-arima_smape_mean,fbp-hw-arima_smape_std,fbp_smape_mean,fbp_smape_std,fbp-arima_smape_mean,fbp-arima_smape_std,fbp-arima_rmse_mean,fbp-arima_rmse_std,fbp-hw-arima_rmse_mean,fbp-hw-arima_rmse_std,...,fbp-hw-arima_coverage_80_mean,fbp-hw-arima_coverage_80_std,fbp_coverage_80_mean,fbp_coverage_80_std,fbp-arima_coverage_95_mean,fbp-arima_coverage_95_std,fbp_coverage_95_mean,fbp_coverage_95_std,fbp-hw-arima_coverage_95_mean,fbp-hw-arima_coverage_95_std
7,3.065333,1.297099,2.905763,1.296214,2.860016,1.118122,75.433452,29.576921,81.607063,35.050051,...,0.830688,0.198254,0.740741,0.217149,0.888889,0.150004,0.899471,0.147266,0.920635,0.144677
14,3.234974,1.175198,2.969147,1.147884,3.046714,1.014237,81.981591,28.428554,89.142651,33.043247,...,0.820106,0.162605,0.740741,0.187049,0.883598,0.127079,0.899471,0.119292,0.920635,0.112649
21,3.296711,0.941638,2.990022,0.913154,3.120898,0.772857,86.005220,24.320027,93.317285,29.024195,...,0.811287,0.133700,0.737213,0.150799,0.881834,0.101764,0.899471,0.096451,0.918871,0.092275
28,3.342359,0.753709,2.992075,0.745672,3.174191,0.575869,88.597525,21.242000,96.077396,26.034720,...,0.816138,0.108374,0.739418,0.129080,0.882275,0.083942,0.900794,0.083071,0.920635,0.080064
35,3.414958,0.659014,3.002447,0.661493,3.244305,0.509347,91.262271,19.704934,99.022850,24.056609,...,0.813757,0.101904,0.739683,0.114102,0.877249,0.075931,0.900529,0.074070,0.919577,0.070002
42,3.481778,0.616122,3.018390,0.627774,3.310223,0.505099,93.523065,18.817474,101.501240,22.721303,...,0.811287,0.096585,0.738977,0.110001,0.869489,0.071542,0.897707,0.069515,0.918871,0.063771
49,3.537259,0.625853,3.039911,0.590333,3.369443,0.491159,95.537065,18.021610,103.572070,22.372135,...,0.811036,0.091722,0.739229,0.105337,0.863946,0.065277,0.894936,0.066421,0.919123,0.059225
56,3.597376,0.606014,3.065053,0.551440,3.422611,0.440157,97.334021,16.474294,105.621674,21.571189,...,0.812169,0.087112,0.737434,0.100139,0.861111,0.057616,0.894180,0.060033,0.918651,0.053648
63,3.665329,0.593982,3.091830,0.524809,3.482404,0.428022,99.321569,15.049107,107.857039,20.216622,...,0.809524,0.083992,0.732510,0.100361,0.857143,0.055686,0.890065,0.056193,0.918283,0.048567
70,3.708069,0.571781,3.111829,0.517341,3.521047,0.407554,100.749661,13.375869,109.465470,18.752636,...,0.807937,0.080553,0.730688,0.101998,0.855026,0.054139,0.887302,0.053207,0.918519,0.045143


## Formatted results

In [9]:
point_forecasts = pd.concat([results_mean_std.filter(like='smape'), 
                             results_mean_std.filter(like='rmse'),
                             results_mean_std.filter(like='mase')], axis=1)

In [10]:
point_forecasts.to_latex(buf='input.tex')

In [11]:
pf_smape = point_forecasts.filter(like='smape')
pf_smape.reindex(sorted(pf_smape.columns), axis=1)

,AR2_smape,arima-fourier_smape,arima_smape,comb_smape,elastic-net_smape,fbp-arima_smape,fbp-hw-arima_smape,fbp_smape,hw-arima_smape,hw_smape,regarima1_smape,regarima_smape,snaive_smape,ssa_smape,tbats_smape
7,3.07 (1.03),2.93 (1.32),3.03 (1.26),4.33 (1.54),3.05 (1.04),2.86 (1.12),3.07 (1.30),2.91 (1.30),3.33 (1.48),2.85 (1.23),3.00 (1.09),3.38 (1.54),3.48 (1.19),3.73 (1.57),3.12 (2.04)
14,3.30 (0.88),3.26 (1.53),3.36 (1.25),4.61 (1.52),3.26 (0.94),3.05 (1.01),3.23 (1.18),2.97 (1.15),3.58 (1.39),3.21 (1.42),3.32 (1.03),3.58 (1.39),3.94 (1.32),3.96 (1.44),3.48 (2.16)
21,3.43 (0.65),3.41 (1.49),3.55 (1.12),4.73 (1.45),3.38 (0.71),3.12 (0.77),3.30 (0.94),2.99 (0.91),3.68 (1.23),3.37 (1.41),3.49 (0.96),3.69 (1.18),4.12 (1.38),4.16 (1.38),3.71 (2.36)
28,3.54 (0.53),3.52 (1.45),3.69 (1.07),4.83 (1.29),3.47 (0.56),3.17 (0.58),3.34 (0.75),2.99 (0.75),3.79 (1.10),3.48 (1.30),3.60 (0.94),3.78 (1.07),4.19 (1.33),4.29 (1.38),3.85 (2.37)
35,3.62 (0.55),3.68 (1.46),3.83 (1.13),5.00 (1.36),3.54 (0.56),3.24 (0.51),3.41 (0.66),3.00 (0.66),3.90 (1.08),3.65 (1.33),3.71 (0.95),3.84 (1.02),4.35 (1.34),4.40 (1.40),4.02 (2.41)
42,3.69 (0.58),3.85 (1.50),3.95 (1.15),5.13 (1.43),3.60 (0.57),3.31 (0.51),3.48 (0.62),3.02 (0.63),4.00 (1.05),3.81 (1.33),3.80 (0.97),3.90 (0.98),4.51 (1.33),4.47 (1.44),4.19 (2.44)
49,3.76 (0.59),3.99 (1.55),4.04 (1.13),5.22 (1.37),3.67 (0.57),3.37 (0.49),3.54 (0.63),3.04 (0.59),4.07 (1.04),3.93 (1.32),3.88 (1.02),3.96 (0.93),4.61 (1.32),4.52 (1.44),4.29 (2.43)
56,3.83 (0.60),4.11 (1.53),4.11 (1.08),5.31 (1.32),3.74 (0.59),3.42 (0.44),3.60 (0.61),3.07 (0.55),4.14 (0.99),4.03 (1.28),3.94 (1.03),4.03 (0.89),4.67 (1.33),4.59 (1.41),4.36 (2.39)
63,3.89 (0.60),4.26 (1.58),4.20 (1.05),5.44 (1.34),3.80 (0.59),3.48 (0.43),3.67 (0.59),3.09 (0.52),4.22 (0.95),4.15 (1.31),4.01 (1.02),4.08 (0.86),4.77 (1.36),4.66 (1.35),4.44 (2.33)
70,3.95 (0.61),4.35 (1.60),4.25 (0.99),5.53 (1.33),3.86 (0.61),3.52 (0.41),3.71 (0.57),3.11 (0.52),4.27 (0.86),4.22 (1.21),4.06 (1.01),4.15 (0.83),4.82 (1.26),4.73 (1.27),4.49 (2.29)


In [12]:
pf_rmse = point_forecasts.filter(like='rmse')
pf_rmse.reindex(sorted(pf_rmse.columns), axis=1)

,AR2_rmse,arima-fourier_rmse,arima_rmse,comb_rmse,elastic-net_rmse,fbp-arima_rmse,fbp-hw-arima_rmse,fbp_rmse,hw-arima_rmse,hw_rmse,regarima1_rmse,regarima_rmse,snaive_rmse,ssa_rmse,tbats_rmse
7,79.65 (26.00),78.96 (39.34),81.17 (39.21),121.15 (41.01),79.48 (26.35),75.43 (29.58),81.61 (35.05),77.86 (34.18),91.22 (44.64),76.79 (36.20),78.33 (28.34),87.92 (38.31),96.91 (37.11),99.68 (45.75),82.98 (52.42)
14,87.84 (24.92),89.20 (45.86),92.20 (41.42),130.12 (42.33),87.30 (25.61),81.98 (28.43),89.14 (33.04),81.54 (29.97),100.95 (43.33),87.59 (42.44),89.08 (30.14),95.40 (36.93),109.71 (41.88),107.77 (43.49),94.06 (57.13)
21,93.01 (21.22),95.51 (44.81),99.07 (39.61),135.53 (40.02),92.36 (21.63),86.01 (24.32),93.32 (29.02),83.81 (25.18),106.54 (41.07),93.53 (42.08),95.31 (29.68),100.00 (34.13),115.38 (42.94),114.16 (41.94),102.13 (60.69)
28,96.70 (19.16),99.51 (43.81),104.11 (38.74),139.21 (37.31),95.89 (19.10),88.60 (21.24),96.08 (26.03),84.85 (21.86),110.55 (39.08),97.80 (39.97),99.46 (29.54),103.14 (32.20),118.34 (42.92),118.58 (41.22),106.99 (60.51)
35,98.97 (18.84),104.95 (43.29),108.75 (39.60),144.06 (38.70),98.02 (18.37),91.26 (19.70),99.02 (24.06),85.83 (19.28),114.72 (38.17),103.12 (39.88),103.17 (29.41),105.37 (30.81),123.26 (42.33),122.19 (40.95),112.48 (60.47)
42,100.84 (18.74),110.06 (43.43),112.57 (39.59),148.10 (40.28),99.76 (18.00),93.52 (18.82),101.50 (22.72),86.66 (17.75),118.10 (37.19),108.01 (39.40),106.21 (29.58),107.24 (29.40),128.11 (40.34),125.00 (41.14),117.29 (60.29)
49,102.52 (18.63),114.38 (43.88),115.77 (39.07),151.58 (38.11),101.38 (17.82),95.54 (18.02),103.57 (22.37),87.59 (16.07),120.79 (36.50),111.69 (38.85),108.79 (30.41),108.97 (28.12),131.40 (39.79),127.36 (41.05),120.61 (59.59)
56,104.45 (18.40),118.15 (42.80),118.62 (38.00),154.70 (36.65),103.33 (17.65),97.33 (16.47),105.62 (21.57),88.68 (14.14),123.42 (35.28),114.82 (38.36),111.11 (30.58),110.98 (26.96),133.95 (40.20),129.83 (40.53),123.02 (58.78)
63,105.98 (17.91),122.53 (42.88),121.62 (36.71),158.46 (35.92),104.91 (17.20),99.32 (15.05),107.86 (20.22),89.69 (12.51),126.33 (33.50),118.45 (38.39),113.54 (30.24),112.63 (25.75),136.59 (40.30),132.14 (39.44),125.67 (57.53)
70,107.67 (17.86),125.80 (43.05),123.99 (35.01),161.25 (34.56),106.70 (17.39),100.75 (13.38),109.47 (18.75),90.46 (11.33),128.43 (30.71),121.14 (35.68),115.57 (29.87),114.47 (24.78),138.70 (37.25),134.65 (37.87),127.89 (55.82)


In [13]:
pf_mase = point_forecasts.filter(like='mase')
pf_mase.reindex(sorted(pf_mase.columns), axis=1)

,AR2_mase,arima-fourier_mase,arima_mase,comb_mase,fbp-arima_mase,fbp-hw-arima_mase,fbp_mase,hw-arima_mase,hw_mase,regarima1_mase,regarima_mase,snaive_mase,ssa_mase,tbats_mase
7,0.82 (0.30),0.88 (0.44),0.82 (0.39),1.17 (0.45),0.77 (0.32),0.83 (0.38),0.78 (0.36),0.91 (0.45),0.77 (0.37),0.81 (0.32),0.91 (0.45),0.94 (0.35),1.01 (0.46),0.85 (0.59)
14,0.89 (0.26),0.98 (0.44),0.91 (0.39),1.24 (0.46),0.82 (0.29),0.88 (0.35),0.80 (0.32),0.97 (0.42),0.87 (0.43),0.90 (0.32),0.96 (0.41),1.06 (0.39),1.07 (0.42),0.95 (0.63)
21,0.92 (0.20),1.05 (0.39),0.96 (0.35),1.28 (0.44),0.84 (0.23),0.90 (0.29),0.81 (0.25),1.00 (0.38),0.92 (0.42),0.94 (0.30),0.99 (0.35),1.12 (0.41),1.13 (0.40),1.01 (0.68)
28,0.95 (0.17),1.11 (0.38),1.00 (0.34),1.31 (0.39),0.86 (0.18),0.91 (0.24),0.81 (0.20),1.03 (0.35),0.95 (0.40),0.98 (0.30),1.02 (0.32),1.14 (0.40),1.16 (0.39),1.05 (0.68)
35,0.97 (0.17),1.16 (0.42),1.04 (0.35),1.35 (0.41),0.88 (0.16),0.93 (0.21),0.81 (0.17),1.06 (0.34),1.00 (0.40),1.01 (0.30),1.04 (0.31),1.18 (0.40),1.19 (0.40),1.10 (0.69)
42,0.99 (0.18),1.21 (0.42),1.07 (0.35),1.39 (0.42),0.90 (0.16),0.95 (0.19),0.82 (0.16),1.09 (0.33),1.04 (0.40),1.03 (0.30),1.05 (0.29),1.23 (0.40),1.21 (0.40),1.15 (0.69)
49,1.01 (0.18),1.24 (0.42),1.10 (0.35),1.41 (0.40),0.91 (0.15),0.96 (0.19),0.82 (0.15),1.11 (0.32),1.07 (0.40),1.05 (0.31),1.07 (0.28),1.25 (0.39),1.23 (0.40),1.18 (0.69)
56,1.03 (0.19),1.28 (0.42),1.12 (0.33),1.44 (0.38),0.93 (0.14),0.98 (0.19),0.83 (0.13),1.13 (0.30),1.10 (0.39),1.07 (0.31),1.09 (0.27),1.27 (0.39),1.25 (0.40),1.19 (0.68)
63,1.05 (0.18),1.31 (0.44),1.14 (0.32),1.48 (0.38),0.95 (0.13),1.00 (0.18),0.84 (0.12),1.16 (0.29),1.13 (0.39),1.09 (0.30),1.10 (0.26),1.30 (0.40),1.27 (0.38),1.22 (0.66)
70,1.06 (0.19),1.34 (0.45),1.16 (0.30),1.50 (0.37),0.96 (0.12),1.01 (0.17),0.84 (0.12),1.17 (0.26),1.15 (0.36),1.10 (0.30),1.12 (0.25),1.31 (0.37),1.29 (0.36),1.23 (0.65)


In [14]:
columns = pf_rmse.columns
columns = [s.replace('_rmse', '') for s in columns]
pf_rmse.columns = columns
pf_rmse.sort_index(axis=1)

,AR2,arima,arima-fourier,comb,elastic-net,fbp,fbp-arima,fbp-hw-arima,hw,hw-arima,regarima,regarima1,snaive,ssa,tbats
7,79.65 (26.00),81.17 (39.21),78.96 (39.34),121.15 (41.01),79.48 (26.35),77.86 (34.18),75.43 (29.58),81.61 (35.05),76.79 (36.20),91.22 (44.64),87.92 (38.31),78.33 (28.34),96.91 (37.11),99.68 (45.75),82.98 (52.42)
14,87.84 (24.92),92.20 (41.42),89.20 (45.86),130.12 (42.33),87.30 (25.61),81.54 (29.97),81.98 (28.43),89.14 (33.04),87.59 (42.44),100.95 (43.33),95.40 (36.93),89.08 (30.14),109.71 (41.88),107.77 (43.49),94.06 (57.13)
21,93.01 (21.22),99.07 (39.61),95.51 (44.81),135.53 (40.02),92.36 (21.63),83.81 (25.18),86.01 (24.32),93.32 (29.02),93.53 (42.08),106.54 (41.07),100.00 (34.13),95.31 (29.68),115.38 (42.94),114.16 (41.94),102.13 (60.69)
28,96.70 (19.16),104.11 (38.74),99.51 (43.81),139.21 (37.31),95.89 (19.10),84.85 (21.86),88.60 (21.24),96.08 (26.03),97.80 (39.97),110.55 (39.08),103.14 (32.20),99.46 (29.54),118.34 (42.92),118.58 (41.22),106.99 (60.51)
35,98.97 (18.84),108.75 (39.60),104.95 (43.29),144.06 (38.70),98.02 (18.37),85.83 (19.28),91.26 (19.70),99.02 (24.06),103.12 (39.88),114.72 (38.17),105.37 (30.81),103.17 (29.41),123.26 (42.33),122.19 (40.95),112.48 (60.47)
42,100.84 (18.74),112.57 (39.59),110.06 (43.43),148.10 (40.28),99.76 (18.00),86.66 (17.75),93.52 (18.82),101.50 (22.72),108.01 (39.40),118.10 (37.19),107.24 (29.40),106.21 (29.58),128.11 (40.34),125.00 (41.14),117.29 (60.29)
49,102.52 (18.63),115.77 (39.07),114.38 (43.88),151.58 (38.11),101.38 (17.82),87.59 (16.07),95.54 (18.02),103.57 (22.37),111.69 (38.85),120.79 (36.50),108.97 (28.12),108.79 (30.41),131.40 (39.79),127.36 (41.05),120.61 (59.59)
56,104.45 (18.40),118.62 (38.00),118.15 (42.80),154.70 (36.65),103.33 (17.65),88.68 (14.14),97.33 (16.47),105.62 (21.57),114.82 (38.36),123.42 (35.28),110.98 (26.96),111.11 (30.58),133.95 (40.20),129.83 (40.53),123.02 (58.78)
63,105.98 (17.91),121.62 (36.71),122.53 (42.88),158.46 (35.92),104.91 (17.20),89.69 (12.51),99.32 (15.05),107.86 (20.22),118.45 (38.39),126.33 (33.50),112.63 (25.75),113.54 (30.24),136.59 (40.30),132.14 (39.44),125.67 (57.53)
70,107.67 (17.86),123.99 (35.01),125.80 (43.05),161.25 (34.56),106.70 (17.39),90.46 (11.33),100.75 (13.38),109.47 (18.75),121.14 (35.68),128.43 (30.71),114.47 (24.78),115.57 (29.87),138.70 (37.25),134.65 (37.87),127.89 (55.82)


In [15]:
columns = pf_smape.columns
columns = [s.replace('_smape', '') for s in columns]
pf_smape.columns = columns
pf_smape.sort_index(axis=1)

,AR2,arima,arima-fourier,comb,elastic-net,fbp,fbp-arima,fbp-hw-arima,hw,hw-arima,regarima,regarima1,snaive,ssa,tbats
7,3.07 (1.03),3.03 (1.26),2.93 (1.32),4.33 (1.54),3.05 (1.04),2.91 (1.30),2.86 (1.12),3.07 (1.30),2.85 (1.23),3.33 (1.48),3.38 (1.54),3.00 (1.09),3.48 (1.19),3.73 (1.57),3.12 (2.04)
14,3.30 (0.88),3.36 (1.25),3.26 (1.53),4.61 (1.52),3.26 (0.94),2.97 (1.15),3.05 (1.01),3.23 (1.18),3.21 (1.42),3.58 (1.39),3.58 (1.39),3.32 (1.03),3.94 (1.32),3.96 (1.44),3.48 (2.16)
21,3.43 (0.65),3.55 (1.12),3.41 (1.49),4.73 (1.45),3.38 (0.71),2.99 (0.91),3.12 (0.77),3.30 (0.94),3.37 (1.41),3.68 (1.23),3.69 (1.18),3.49 (0.96),4.12 (1.38),4.16 (1.38),3.71 (2.36)
28,3.54 (0.53),3.69 (1.07),3.52 (1.45),4.83 (1.29),3.47 (0.56),2.99 (0.75),3.17 (0.58),3.34 (0.75),3.48 (1.30),3.79 (1.10),3.78 (1.07),3.60 (0.94),4.19 (1.33),4.29 (1.38),3.85 (2.37)
35,3.62 (0.55),3.83 (1.13),3.68 (1.46),5.00 (1.36),3.54 (0.56),3.00 (0.66),3.24 (0.51),3.41 (0.66),3.65 (1.33),3.90 (1.08),3.84 (1.02),3.71 (0.95),4.35 (1.34),4.40 (1.40),4.02 (2.41)
42,3.69 (0.58),3.95 (1.15),3.85 (1.50),5.13 (1.43),3.60 (0.57),3.02 (0.63),3.31 (0.51),3.48 (0.62),3.81 (1.33),4.00 (1.05),3.90 (0.98),3.80 (0.97),4.51 (1.33),4.47 (1.44),4.19 (2.44)
49,3.76 (0.59),4.04 (1.13),3.99 (1.55),5.22 (1.37),3.67 (0.57),3.04 (0.59),3.37 (0.49),3.54 (0.63),3.93 (1.32),4.07 (1.04),3.96 (0.93),3.88 (1.02),4.61 (1.32),4.52 (1.44),4.29 (2.43)
56,3.83 (0.60),4.11 (1.08),4.11 (1.53),5.31 (1.32),3.74 (0.59),3.07 (0.55),3.42 (0.44),3.60 (0.61),4.03 (1.28),4.14 (0.99),4.03 (0.89),3.94 (1.03),4.67 (1.33),4.59 (1.41),4.36 (2.39)
63,3.89 (0.60),4.20 (1.05),4.26 (1.58),5.44 (1.34),3.80 (0.59),3.09 (0.52),3.48 (0.43),3.67 (0.59),4.15 (1.31),4.22 (0.95),4.08 (0.86),4.01 (1.02),4.77 (1.36),4.66 (1.35),4.44 (2.33)
70,3.95 (0.61),4.25 (0.99),4.35 (1.60),5.53 (1.33),3.86 (0.61),3.11 (0.52),3.52 (0.41),3.71 (0.57),4.22 (1.21),4.27 (0.86),4.15 (0.83),4.06 (1.01),4.82 (1.26),4.73 (1.27),4.49 (2.29)


In [16]:
columns = pf_mase.columns
columns = [s.replace('_mase', '') for s in columns]
pf_mase.columns = columns
pf_mase.sort_index(axis=1)

,AR2,arima,arima-fourier,comb,fbp,fbp-arima,fbp-hw-arima,hw,hw-arima,regarima,regarima1,snaive,ssa,tbats
7,0.82 (0.30),0.82 (0.39),0.88 (0.44),1.17 (0.45),0.78 (0.36),0.77 (0.32),0.83 (0.38),0.77 (0.37),0.91 (0.45),0.91 (0.45),0.81 (0.32),0.94 (0.35),1.01 (0.46),0.85 (0.59)
14,0.89 (0.26),0.91 (0.39),0.98 (0.44),1.24 (0.46),0.80 (0.32),0.82 (0.29),0.88 (0.35),0.87 (0.43),0.97 (0.42),0.96 (0.41),0.90 (0.32),1.06 (0.39),1.07 (0.42),0.95 (0.63)
21,0.92 (0.20),0.96 (0.35),1.05 (0.39),1.28 (0.44),0.81 (0.25),0.84 (0.23),0.90 (0.29),0.92 (0.42),1.00 (0.38),0.99 (0.35),0.94 (0.30),1.12 (0.41),1.13 (0.40),1.01 (0.68)
28,0.95 (0.17),1.00 (0.34),1.11 (0.38),1.31 (0.39),0.81 (0.20),0.86 (0.18),0.91 (0.24),0.95 (0.40),1.03 (0.35),1.02 (0.32),0.98 (0.30),1.14 (0.40),1.16 (0.39),1.05 (0.68)
35,0.97 (0.17),1.04 (0.35),1.16 (0.42),1.35 (0.41),0.81 (0.17),0.88 (0.16),0.93 (0.21),1.00 (0.40),1.06 (0.34),1.04 (0.31),1.01 (0.30),1.18 (0.40),1.19 (0.40),1.10 (0.69)
42,0.99 (0.18),1.07 (0.35),1.21 (0.42),1.39 (0.42),0.82 (0.16),0.90 (0.16),0.95 (0.19),1.04 (0.40),1.09 (0.33),1.05 (0.29),1.03 (0.30),1.23 (0.40),1.21 (0.40),1.15 (0.69)
49,1.01 (0.18),1.10 (0.35),1.24 (0.42),1.41 (0.40),0.82 (0.15),0.91 (0.15),0.96 (0.19),1.07 (0.40),1.11 (0.32),1.07 (0.28),1.05 (0.31),1.25 (0.39),1.23 (0.40),1.18 (0.69)
56,1.03 (0.19),1.12 (0.33),1.28 (0.42),1.44 (0.38),0.83 (0.13),0.93 (0.14),0.98 (0.19),1.10 (0.39),1.13 (0.30),1.09 (0.27),1.07 (0.31),1.27 (0.39),1.25 (0.40),1.19 (0.68)
63,1.05 (0.18),1.14 (0.32),1.31 (0.44),1.48 (0.38),0.84 (0.12),0.95 (0.13),1.00 (0.18),1.13 (0.39),1.16 (0.29),1.10 (0.26),1.09 (0.30),1.30 (0.40),1.27 (0.38),1.22 (0.66)
70,1.06 (0.19),1.16 (0.30),1.34 (0.45),1.50 (0.37),0.84 (0.12),0.96 (0.12),1.01 (0.17),1.15 (0.36),1.17 (0.26),1.12 (0.25),1.10 (0.30),1.31 (0.37),1.29 (0.36),1.23 (0.65)


In [17]:
coverage = results_mean_std.filter(like='80')
coverage

,fbp-arima_coverage_80,ssa_coverage_80,AR2_coverage_80,arima-fourier_coverage_80,regarima_coverage_80,elastic-net_coverage_80,tbats_coverage_80,arima_coverage_80,hw_coverage_80,regarima1_coverage_80,comb_coverage_80,hw-arima_coverage_80,fbp-hw-arima_coverage_80,fbp_coverage_80
7,0.78 (0.21),0.85 (0.22),0.71 (0.20),0.90 (0.13),0.69 (0.26),0.72 (0.21),0.86 (0.16),0.82 (0.20),0.86 (0.17),0.79 (0.20),0.83 (0.14),0.84 (0.18),0.83 (0.20),0.74 (0.22)
14,0.77 (0.17),0.83 (0.21),0.66 (0.17),0.89 (0.16),0.67 (0.21),0.68 (0.17),0.87 (0.16),0.79 (0.16),0.86 (0.17),0.75 (0.17),0.91 (0.09),0.83 (0.16),0.82 (0.16),0.74 (0.19)
21,0.76 (0.14),0.81 (0.19),0.65 (0.12),0.91 (0.14),0.66 (0.17),0.66 (0.12),0.89 (0.14),0.78 (0.14),0.87 (0.16),0.73 (0.15),0.93 (0.06),0.82 (0.15),0.81 (0.13),0.74 (0.15)
28,0.77 (0.11),0.80 (0.20),0.63 (0.09),0.92 (0.13),0.65 (0.14),0.65 (0.10),0.90 (0.12),0.77 (0.13),0.89 (0.14),0.72 (0.13),0.95 (0.05),0.82 (0.13),0.82 (0.11),0.74 (0.13)
35,0.76 (0.10),0.79 (0.20),0.62 (0.10),0.92 (0.12),0.64 (0.13),0.64 (0.10),0.91 (0.11),0.75 (0.13),0.89 (0.14),0.71 (0.13),0.95 (0.05),0.82 (0.12),0.81 (0.10),0.74 (0.11)
42,0.75 (0.10),0.79 (0.20),0.61 (0.11),0.93 (0.12),0.63 (0.12),0.63 (0.10),0.92 (0.10),0.74 (0.14),0.90 (0.13),0.71 (0.14),0.96 (0.04),0.82 (0.11),0.81 (0.10),0.74 (0.11)
49,0.75 (0.09),0.79 (0.20),0.60 (0.11),0.93 (0.10),0.63 (0.11),0.62 (0.11),0.93 (0.09),0.74 (0.14),0.91 (0.12),0.70 (0.14),0.97 (0.03),0.83 (0.11),0.81 (0.09),0.74 (0.11)
56,0.74 (0.08),0.79 (0.19),0.58 (0.11),0.93 (0.09),0.62 (0.11),0.61 (0.11),0.93 (0.08),0.73 (0.13),0.91 (0.10),0.70 (0.14),0.97 (0.03),0.83 (0.10),0.81 (0.09),0.74 (0.10)
63,0.73 (0.07),0.79 (0.18),0.57 (0.11),0.94 (0.08),0.61 (0.10),0.60 (0.11),0.93 (0.07),0.72 (0.13),0.92 (0.10),0.70 (0.14),0.97 (0.03),0.83 (0.10),0.81 (0.08),0.73 (0.10)
70,0.73 (0.07),0.79 (0.17),0.57 (0.11),0.94 (0.08),0.60 (0.09),0.59 (0.11),0.94 (0.06),0.72 (0.12),0.92 (0.09),0.69 (0.14),0.97 (0.02),0.83 (0.09),0.81 (0.08),0.73 (0.10)


In [18]:
coverage_80 = results_mean_std.filter(like='80')
columns = coverage_80.columns
columns = [s.replace('_coverage_80', '') for s in columns]
coverage_80.columns = columns
coverage_80.sort_index(axis=1)

,AR2,arima,arima-fourier,comb,elastic-net,fbp,fbp-arima,fbp-hw-arima,hw,hw-arima,regarima,regarima1,ssa,tbats
7,0.71 (0.20),0.82 (0.20),0.90 (0.13),0.83 (0.14),0.72 (0.21),0.74 (0.22),0.78 (0.21),0.83 (0.20),0.86 (0.17),0.84 (0.18),0.69 (0.26),0.79 (0.20),0.85 (0.22),0.86 (0.16)
14,0.66 (0.17),0.79 (0.16),0.89 (0.16),0.91 (0.09),0.68 (0.17),0.74 (0.19),0.77 (0.17),0.82 (0.16),0.86 (0.17),0.83 (0.16),0.67 (0.21),0.75 (0.17),0.83 (0.21),0.87 (0.16)
21,0.65 (0.12),0.78 (0.14),0.91 (0.14),0.93 (0.06),0.66 (0.12),0.74 (0.15),0.76 (0.14),0.81 (0.13),0.87 (0.16),0.82 (0.15),0.66 (0.17),0.73 (0.15),0.81 (0.19),0.89 (0.14)
28,0.63 (0.09),0.77 (0.13),0.92 (0.13),0.95 (0.05),0.65 (0.10),0.74 (0.13),0.77 (0.11),0.82 (0.11),0.89 (0.14),0.82 (0.13),0.65 (0.14),0.72 (0.13),0.80 (0.20),0.90 (0.12)
35,0.62 (0.10),0.75 (0.13),0.92 (0.12),0.95 (0.05),0.64 (0.10),0.74 (0.11),0.76 (0.10),0.81 (0.10),0.89 (0.14),0.82 (0.12),0.64 (0.13),0.71 (0.13),0.79 (0.20),0.91 (0.11)
42,0.61 (0.11),0.74 (0.14),0.93 (0.12),0.96 (0.04),0.63 (0.10),0.74 (0.11),0.75 (0.10),0.81 (0.10),0.90 (0.13),0.82 (0.11),0.63 (0.12),0.71 (0.14),0.79 (0.20),0.92 (0.10)
49,0.60 (0.11),0.74 (0.14),0.93 (0.10),0.97 (0.03),0.62 (0.11),0.74 (0.11),0.75 (0.09),0.81 (0.09),0.91 (0.12),0.83 (0.11),0.63 (0.11),0.70 (0.14),0.79 (0.20),0.93 (0.09)
56,0.58 (0.11),0.73 (0.13),0.93 (0.09),0.97 (0.03),0.61 (0.11),0.74 (0.10),0.74 (0.08),0.81 (0.09),0.91 (0.10),0.83 (0.10),0.62 (0.11),0.70 (0.14),0.79 (0.19),0.93 (0.08)
63,0.57 (0.11),0.72 (0.13),0.94 (0.08),0.97 (0.03),0.60 (0.11),0.73 (0.10),0.73 (0.07),0.81 (0.08),0.92 (0.10),0.83 (0.10),0.61 (0.10),0.70 (0.14),0.79 (0.18),0.93 (0.07)
70,0.57 (0.11),0.72 (0.12),0.94 (0.08),0.97 (0.02),0.59 (0.11),0.73 (0.10),0.73 (0.07),0.81 (0.08),0.92 (0.09),0.83 (0.09),0.60 (0.09),0.69 (0.14),0.79 (0.17),0.94 (0.06)


In [19]:
coverage_95 = results_mean_std.filter(like='95')
columns = coverage_95.columns
columns = [s.replace('_coverage_95', '') for s in columns]
coverage_95.columns = columns
coverage_95.sort_index(axis=1)

,AR2,arima,arima-fourier,comb,fbp,fbp-arima,fbp-hw-arima,hw,hw-arima,regarima,regarima1,ssa,tbats
7,0.92 (0.14),0.96 (0.09),0.98 (0.06),0.96 (0.08),0.90 (0.15),0.89 (0.15),0.92 (0.14),0.97 (0.07),0.95 (0.10),0.90 (0.18),0.93 (0.12),0.90 (0.19),0.97 (0.06)
14,0.89 (0.13),0.95 (0.11),0.97 (0.09),0.97 (0.05),0.90 (0.12),0.88 (0.13),0.92 (0.11),0.96 (0.11),0.95 (0.09),0.88 (0.17),0.93 (0.11),0.91 (0.16),0.97 (0.08)
21,0.87 (0.10),0.95 (0.09),0.97 (0.07),0.98 (0.04),0.90 (0.10),0.88 (0.10),0.92 (0.09),0.96 (0.11),0.94 (0.10),0.87 (0.15),0.93 (0.10),0.91 (0.15),0.97 (0.06)
28,0.85 (0.10),0.95 (0.09),0.98 (0.05),0.99 (0.03),0.90 (0.08),0.88 (0.08),0.92 (0.08),0.97 (0.08),0.95 (0.08),0.87 (0.13),0.93 (0.09),0.91 (0.14),0.98 (0.05)
35,0.84 (0.09),0.94 (0.08),0.98 (0.04),0.99 (0.02),0.90 (0.07),0.88 (0.08),0.92 (0.07),0.97 (0.07),0.94 (0.08),0.86 (0.12),0.92 (0.08),0.91 (0.14),0.98 (0.04)
42,0.84 (0.09),0.94 (0.08),0.98 (0.03),0.99 (0.02),0.90 (0.07),0.87 (0.07),0.92 (0.06),0.98 (0.06),0.94 (0.07),0.85 (0.11),0.91 (0.08),0.91 (0.14),0.98 (0.03)
49,0.83 (0.09),0.93 (0.08),0.98 (0.03),0.99 (0.02),0.89 (0.07),0.86 (0.07),0.92 (0.06),0.98 (0.05),0.93 (0.07),0.85 (0.11),0.91 (0.08),0.91 (0.14),0.98 (0.03)
56,0.82 (0.09),0.93 (0.08),0.98 (0.03),0.99 (0.01),0.89 (0.06),0.86 (0.06),0.92 (0.05),0.98 (0.04),0.93 (0.07),0.84 (0.10),0.91 (0.08),0.91 (0.14),0.98 (0.02)
63,0.81 (0.09),0.93 (0.07),0.98 (0.02),0.99 (0.01),0.89 (0.06),0.86 (0.06),0.92 (0.05),0.98 (0.04),0.94 (0.06),0.83 (0.09),0.90 (0.07),0.91 (0.12),0.99 (0.02)
70,0.80 (0.09),0.93 (0.06),0.99 (0.02),0.99 (0.01),0.89 (0.05),0.86 (0.05),0.92 (0.05),0.98 (0.03),0.94 (0.06),0.83 (0.09),0.90 (0.07),0.91 (0.11),0.99 (0.02)


In [20]:
#Write these rather large tables to a latex file to use in appendix

#Table S1: MASE
pf_mase.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}tableS1.tex')

#Table S2: 80% Coverage
coverage_80.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}tableS2.tex')

#Table S3: 95% coverage
coverage_95.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}tableS3.tex')

#extra rables for sMAPE and RMSE - not used in paper.
pf_smape.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}symmetricmape.tex')
pf_rmse.sort_index(axis=1).to_latex(buf=f'{APPENDIX_PATH}rootmeansqua.tex')


print(f'Appendix summary tables written to {APPENDIX_PATH}')

## Naive Benchmark (Seasonal Naive)

A naive baseline forecasting method was chosen. This was to ensure that the sophisticated methods we test in the study were only considered for the final benchmark if they provided more more accurate point forecasts than the simplest of models. As emergency care demand data are seasonal we opted for the well-known Seasonal Naive method.  This method works by using the most recent observation for the same day and carrying it forward.  For example, if we are forecasting next Tuesday then the observation from the most recent Tuesday is used as the predicted value.

In [21]:
pf_snaive = point_forecasts.filter(like='snaive')
pf_snaive.reindex(sorted(pf_snaive.columns), axis=1)
columns = pf_snaive.columns
columns = [s.replace('_rmse', '') for s in columns]
columns = [s.replace('_smape', '') for s in columns]
columns = [s.replace('_mase', '') for s in columns]
pf_snaive.columns = ['sMAPE', 'RMSE', 'MASE']
pf_snaive[:-1].to_latex(buf='snaive.tex')

In [22]:
pf_snaive[['MASE', 'sMAPE', 'RMSE']].to_latex(buf=f'{TABLE_PATH}table2.tex')

In [23]:
pf_snaive[:-1]

,sMAPE,RMSE,MASE
7,3.48 (1.19),96.91 (37.11),0.94 (0.35)
14,3.94 (1.32),109.71 (41.88),1.06 (0.39)
21,4.12 (1.38),115.38 (42.94),1.12 (0.41)
28,4.19 (1.33),118.34 (42.92),1.14 (0.40)
35,4.35 (1.34),123.26 (42.33),1.18 (0.40)
42,4.51 (1.33),128.11 (40.34),1.23 (0.40)
49,4.61 (1.32),131.40 (39.79),1.25 (0.39)
56,4.67 (1.33),133.95 (40.20),1.27 (0.39)
63,4.77 (1.36),136.59 (40.30),1.30 (0.40)
70,4.82 (1.26),138.70 (37.25),1.31 (0.37)


In [26]:
print('\nTable 2: Cross-Validation of Seasonal Naive Point Forecasts')
print(pf_snaive[:-1])


Table 2: Cross-Validation of Seasonal Naive Point Forecasts
          sMAPE            RMSE         MASE
7   3.48 (1.19)   96.91 (37.11)  0.94 (0.35)
14  3.94 (1.32)  109.71 (41.88)  1.06 (0.39)
21  4.12 (1.38)  115.38 (42.94)  1.12 (0.41)
28  4.19 (1.33)  118.34 (42.92)  1.14 (0.40)
35  4.35 (1.34)  123.26 (42.33)  1.18 (0.40)
42  4.51 (1.33)  128.11 (40.34)  1.23 (0.40)
49  4.61 (1.32)  131.40 (39.79)  1.25 (0.39)
56  4.67 (1.33)  133.95 (40.20)  1.27 (0.39)
63  4.77 (1.36)  136.59 (40.30)  1.30 (0.40)
70  4.82 (1.26)  138.70 (37.25)  1.31 (0.37)
77  4.88 (1.18)  140.73 (34.49)  1.33 (0.34)
84  4.94 (1.21)  143.04 (34.07)  1.34 (0.35)
